In [ ]:
!pip install numpy matplotlib opencv-python

In [ ]:
import pandas as pd
import sqlite3
import requests
import matplotlib.pyplot as plt
import numpy as np
import cv2
from matplotlib.colors import ListedColormap

# Load Data

In [ ]:
def load_sqlite_to_dataframe(sqlite_file, table_name):
    # Connect to the SQLite database
    conn = sqlite3.connect(sqlite_file)
    # Load data into a DataFrame
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    # Close the connection
    conn.close()
    return df

In [ ]:
# GH data
df_metrics_gh = load_sqlite_to_dataframe('data/GH/tracking.db', 'metrics')
df_metrics_gh['filter'] = 'GH'
df_tracks_gh = load_sqlite_to_dataframe('data/GH/tracking.db', 'tracks')
df_tracks_gh['filter'] = 'GH'

In [ ]:
# Kalman data
df_metrics_kalman = load_sqlite_to_dataframe('data/Kalman/tracking.db', 'metrics')
df_metrics_kalman['filter'] = 'Kalman'
df_tracks_kalman = load_sqlite_to_dataframe('data/Kalman/tracking.db', 'tracks')
df_tracks_kalman['filter'] = 'Kalman'

In [ ]:
# Particles data
df_metrics_particles = load_sqlite_to_dataframe('data/Particles/tracking.db', 'metrics')
df_metrics_particles['filter'] = 'Particles'
df_tracks_particles = load_sqlite_to_dataframe('data/Particles/tracking.db', 'tracks')
df_tracks_particles['filter'] = 'Particles'

In [ ]:
# Unscented data
df_metrics_unscented = load_sqlite_to_dataframe('data/Unscented/tracking.db', 'metrics')
df_metrics_unscented['filter'] = 'Unscented'
df_tracks_unscented = load_sqlite_to_dataframe('data/Unscented/tracking.db', 'tracks')
df_tracks_unscented['filter'] = 'Unscented'

In [ ]:
df_metrics = pd.concat([df_metrics_gh, df_metrics_kalman, df_metrics_unscented, df_metrics_particles])
df_metrics = df_metrics.reset_index().drop('index', axis=1)

df_metrics['timestamp'] = pd.to_datetime(df_metrics['timestamp'])
df_metrics['value'] = df_metrics['value'].astype(float)

df_metrics

In [ ]:
df_tracks = pd.concat([df_tracks_gh, df_tracks_kalman, df_tracks_unscented, df_tracks_particles])
df_tracks = df_tracks.reset_index().drop('index', axis=1)

df_tracks['timestamp'] = df_tracks['timestamp'].astype(int)

df_tracks['score'] = df_tracks['score'].apply(lambda x: None if isinstance(x, bytes) else x)
df_tracks['score'] = df_tracks['score'].astype(float)
#df_tracks[df_tracks['tracker'] == '0224473a-65a5-43e8-b700-a186ca32bdaf']
df_tracks.head()

# FPS

In [ ]:
df_fps = df_metrics[(df_metrics['metric'] == 'FPS') & (df_metrics['frame'] > 1)]
df_fps = df_fps.reset_index().drop('index', axis=1)
df_fps['detection'] = df_fps['frame'] % 2 == 0

def plot_fps(df, detection: int):
  df = df.copy()
  df = df[df['detection'] == detection].reset_index().drop('index', axis=1)

  # Plotting the data
  plt.figure(figsize=(10, 6))

  # Group by 'filter' and plot each group
  for key, grp in df.groupby('filter'):
    print(key + ': ' + str(np.mean(grp['value'])))
    plt.plot(grp['frame'], grp['value'], label=key)

  plt.xlabel('Frame')
  plt.ylabel('FPS')
  plt.title(f'FPS by Filter (detection={detection})')
  plt.legend(title='Filter')
  plt.grid(True)
  plt.show()


plot_fps(df_fps, 0)
plot_fps(df_fps, 1)

Observando la gran diferencia de velocidades en ambos escenarios es notable que la detección genera un procesamiento muy lento. Luego, observando el escenario sin detección (solo predicción), GH es el más rápido, seguido por Kalman, Particles y, por último, Unscented.

Medias:
* GH: 3666.153846153846
* Kalman: 1153.2662721893491
* Particles: 365.94674556213016
* Unscented: 88.38461538461539

# Detections

In [ ]:
df_detections = df_metrics[df_metrics['metric'] == 'Detections']
df_detections = df_detections.reset_index().drop('index', axis=1)
df_detections['detection'] = df_detections['frame'] % 2 == 0

# Plotting the data
plt.figure(figsize=(10, 6))

# Group by 'filter' and plot each group
for key, grp in df_detections.groupby('filter'):
  plt.plot(grp['frame'], grp['value'], label=key)

plt.xlabel('Frame')
plt.ylabel('Detections')
plt.title(f'Detections by Filter')
plt.legend(title='Filter')
plt.grid(True)
plt.show()


Es esperado que la metrica de objetos detectados sea exactamente igual en todas las pruebas dado que esta etapa es independiente. Con este gráfico confirmamos que el modelo de detecciones no tuvo diferencias entre las ejecuciones.

# Active Trackers

In [ ]:
df_active = df_metrics[(df_metrics['metric'] == 'Active Trackers') & (df_metrics['frame'] > 1)]
df_active = df_active.reset_index().drop('index', axis=1)

# Plotting the data
plt.figure(figsize=(10, 6))

# Group by 'filter' and plot each group
for key, grp in df_active.groupby('filter'):
  plt.plot(grp['frame'], grp['value'], label=key)

plt.xlabel('Frame')
plt.ylabel('Active Trackers')
plt.title(f'Active Trackers by Filter')
plt.legend(title='Filter')
plt.grid(True)
plt.show()

Estamos generando erróneamente más trackers en Particles. Eso significa que las predicciones no están bien y se generan nuevos trackers en las detecciones. Kalman y Unscented Kalman tiene el mismo comportamiento.

# Trackers Delta

In [ ]:
df_delta = df_metrics[(df_metrics['metric'] == 'Trackers Delta')]
df_delta = df_delta.reset_index().drop('index', axis=1)

# Plotting the data
plt.figure(figsize=(10, 6))

# Group by 'filter' and plot each group
for key, grp in df_delta.groupby('filter'):
  plt.plot(grp['frame'], grp['value'], label=key)

plt.xlabel('Frame')
plt.ylabel('Trackers Delta')
plt.title(f'Trackers Delta by Filter')
plt.legend(title='Filter')
plt.grid(True)
plt.show()

Por lo comentado mas arriba (sobre los trackers activos de Particles) es esperable que haya mayor movimiento (generación y eliminación) de trackers.

# Prediction Errors

In [ ]:
df_errors = df_metrics[(df_metrics['metric'] == 'Tracker Errors') & (df_metrics['frame'] > 1) & (df_metrics['frame'] % 2 == 0)]
df_errors = df_errors.reset_index().drop('index', axis=1)

# Plotting the data
plt.figure(figsize=(10, 6))

# Group by 'filter' and plot each group
for key, grp in df_errors.groupby('filter'):
  plt.plot(grp['frame'], grp['value'], label=key)

plt.xlabel('Frame')
plt.ylabel('Tracker Prediction Errors')
plt.title(f'Tracker Prediction Errors by Filter')
plt.legend(title='Filter')
plt.grid(True)
plt.show()

Claramente las precicciones realizadas por Particles son mejores. Kalman y Unscented Kalman no presentan diferencias. Es esperable que GH sea el pero dada la simpleza del modelo.

# Tracker Lifetime

In [ ]:
# Group by 'filter' and 'tracker' and calculate min, max, and lifetime
df_lifetime = df_tracks.groupby(['filter', 'tracker']).agg(
    min_timestamp=('timestamp', 'min'),
    max_timestamp=('timestamp', 'max'),
)

df_lifetime['lifetime'] = (df_lifetime['max_timestamp'] - df_lifetime['min_timestamp']) / 1000000000 #seconds
df_lifetime = df_lifetime.reset_index()

# Create a dictionary to store data for each filter
filter_data = {}
for f in df_lifetime['filter'].unique():
    filter_data[f] = df_lifetime[df_lifetime['filter'] == f]['lifetime']

# Plot all boxplots on the same figure
plt.figure(figsize=(10, 6))
plt.boxplot(filter_data.values(), labels=filter_data.keys())
plt.xlabel('Filter')
plt.ylabel('Lifetime (seconds)')
plt.title('Boxplot of Tracker Lifetime by Filter')
plt.grid(True)
plt.show()

Unscented tiene un mayor tiempo de vida de los trackers, eso significaría que sigue durante más tiempo a las personas. No tenemos un baseline para esta métrica, pero entendemos que mientras más tiempo estuvo vivo el tracker, mejor fue la predicción (no fue descartado por mala predicción).

El bajo tiempo de vida de los trackers de Particles confirma lo expuesto antes, las predicciones son malas y los trackers están continuamente descartandose.

# Detection Score

In [ ]:
df_scores = df_tracks.copy()

# Agrupar por celda y calcular la media del score
df_scores_grouped = df_scores[['cell','score']].groupby('cell').mean().reset_index()

# Tamaño de la imagen
img_width, img_height = 1280, 720

# Tamaño de las celdas del heatmap
cell_size = 40

# Ruta de la imagen en el disco
image_path = 'assets/frame.jpg'

# Cargar la imagen desde el disco
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convertir de BGR a RGB para matplotlib
img_height, img_width = image.shape[:2]

# Convertir el dataframe agrupado en una matriz de datos para el heatmap
heatmap_data = np.zeros((img_height // cell_size, img_width // cell_size))
'''
for index, row in df_scores_grouped.iterrows():
    cell = int(row['cell'])
    score = float(row['score'])
    j, i = cell // (img_width // cell_size), cell % (img_width // cell_size)
    heatmap_data[i, j] = score
'''
# Crear el heatmap con matplotlib
fig, ax = plt.subplots(figsize=(16, 9))  # Aumentar el tamaño de la figura
ax.imshow(image, alpha=1)
cax = ax.imshow(heatmap_data, cmap='brg', alpha=0.5, extent=[0, img_width, img_height, 0])  # Ajustar la extensión

# Añadir colorbar
fig.colorbar(cax)

plt.title('Detection Score by Cell')
plt.axis('off')  # Desactivar los ejes

plt.show()

Tenemos un umbral de 0.5 establecido en el programa, debajo de eso es ignorado. En este grafico, lo mejor es tener verde (altos scores de detección).

# Cell Traffic

In [ ]:
df_traffic = df_tracks.copy()

# Agrupar por celda y calcular la media del score
df_traffic_grouped = df_traffic.groupby('cell').size().reset_index(name='score')

# Tamaño de la imagen
img_width, img_height = 1280, 720

# Tamaño de las celdas del heatmap
cell_size = 40

# Ruta de la imagen en el disco
image_path = 'assets/frame.jpg'

# Cargar la imagen desde el disco
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convertir de BGR a RGB para matplotlib

# Convertir el dataframe agrupado en una matriz de datos para el heatmap
heatmap_data = np.zeros((img_height // cell_size, img_width // cell_size))
'''
for index, row in df_traffic_grouped.iterrows():
    cell = int(row['cell'])
    score = float(row['score'])
    i, j = cell // (img_width // cell_size), cell % (img_width // cell_size)
    heatmap_data[i, j] = score
'''
# Crear el heatmap con matplotlib
fig, ax = plt.subplots(figsize=(16, 9))  # Aumentar el tamaño de la figura
ax.imshow(image, alpha=1)
cax = ax.imshow(heatmap_data, cmap='Oranges', alpha=0.5, extent=[0, img_width, img_height, 0])  # Ajustar la extensión

# Añadir colorbar
fig.colorbar(cax)

plt.title('Traffic by Cell')
plt.axis('off')  # Desactivar los ejes

plt.show()